In [1]:
# 01 - Module Imports

from datetime import datetime
from datetime import date
import os
import pandas as pd

from config import engine
from openpyxl import load_workbook
from sqlalchemy import create_engine

In [2]:
# 02 - Data Import - Equipment Usage

import_file = 'TestData/Eq Usage Post - 2020.xlsx'

import_df_raw = pd.read_excel(import_file)
import_df = import_df_raw[[ 'equipment_no', 'transaction_no','date_booked', 'job_no', 'equipment_units', 'cost_code_no']].copy()

today = date.today()
import_df['date_modified'] = today

C:\Users\agreg\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
 #03 - Clean Up Import Data

import_df['job_no'] = import_df['job_no'].astype(str)
import_df['cost_code_no'] = import_df['cost_code_no'].astype(str)
import_df['cost_code_no'] = import_df['cost_code_no'].apply(lambda x: x.replace(' ', ''))
import_df['equipment_no'] = import_df['equipment_no'].apply(lambda x: x.replace(' ', ''))
import_df['date_booked'] = import_df['date_booked'].dt.date

In [4]:
# 04 - Query Transaction Numbers that Exist in the DB

transaction_df_raw = pd.read_sql_table('eq_usage', engine)
transaction_df = transaction_df_raw[['transaction_no']].copy()
transaction_list = transaction_df['transaction_no'].unique()

In [5]:
#05 - Remove import_df Rows with Transaction Numbers that Appear in the Database

rows_to_drop = import_df['transaction_no'].isin(transaction_df.loc[:, 'transaction_no'])
import_df = import_df[~rows_to_drop]
print(len(import_df))


0


In [6]:
#06 - Add New Transactions to Database

import_df.to_sql('eq_usage', engine, if_exists='append', index=False)